pip install --force-reinstall openai==1.8

In [5]:
from datasets import load_dataset
import pandas as pd
from rouge import Rouge
from openai import OpenAI
import json
from settings import *
from utils import preprocess, prompt, score, utils
import os 

from langchain.llms import OpenAI
from langchain import HuggingFaceHub, LLMChain
from langchain.prompts import load_prompt, PromptTemplate
from tqdm import tqdm

from langchain.prompts import PromptTemplate
from langchain import LLMChain
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
import warnings
warnings.filterwarnings("ignore")
                                                             
config = utils.load_json(CONFIG_DIR)

In [7]:
pubmed = utils.load_data(config['data_name'])
pubmed

,article,abstract
14,a review of the literature and an extensive me...,backgrounda review of the literature and an ex...
53,"nathan , as an oncology fellow , knew well tha...",t cells tell macrophages when to start making ...
61,temporary henna tattoos or pseudotattoo have b...,temporary henna tattoos or pseudotattoos have ...
71,care coordination is an important aspect of nu...,introductioncare coordination is an important ...
123,the laparoscopic removal of a cervical stump f...,"a 43-year - old , who underwent a subtotal hys..."
...,...,...
119684,to assess attitudes and practices of documenta...,purposes : to assess attitudes and practices o...
119764,"hearts , isolated from wildtype zebrafish embr...",electrical gradients are critical for many bio...
119802,both f-18 fluorodeoxyglucose ( fdg ) and c-11 ...,"a 10-year - boy post - operative , post - radi..."
119867,monilethrix is an autosomal dominant disorder ...,congenital hypotrichosis may be due to a numbe...


In [6]:
# os.environ['HUGGINGFACEHUB_API_TOKEN'] = load_json(API_KEY)['huggingface_key']
# summarizer = HuggingFaceHub(
#     repo_id="gpt2",
#     model_kwargs={"temperature":0.2}
# )
# template = """
# You are an abstractive summarizer that follows the output pattern.
# Please revise the extracted summary based on the document. The revised summary should include the information in the extracted summary. Docu- ment: {document} Extractive Summary: [Extractive Summary] [Format Instruction].
# """

# response_list = []
# for idx in tqdm(range(10)):
#     doc = filter_pubmed.iloc[idx, 0]
#     prompt = PromptTemplate(template=template, input_variables=["document"])
#     llm_chain = LLMChain(prompt=prompt, llm=summarizer)
#     response = llm_chain.invoke(input = doc)['document']
#     if len(response) > 0:
#         response_list.append([response, filter_pubmed.iloc[idx, 1]])
    

/opt/conda/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)
100%|██████████| 10/10 [00:02<00:00,  3.62it/s]


In [8]:
MODEL_NAME = "mistralai/Mistral-7B-v0.1"
cache_dir = "/data/ephemeral/Youtube-Short-Generator/mistral"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME,cache_dir=cache_dir)
tokenizer.pad_token_id = tokenizer.eos_token_id
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME,cache_dir=cache_dir)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10, device = 0, pad_token_id=tokenizer.eos_token_id)
hf = HuggingFacePipeline(pipeline=pipe)

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [9]:
template = """
You are an abstractive summarizer that follows the output pattern.
Please revise the extracted summary based on the document. The revised summary should include the information in the extracted summary. Docu- ment: {document} Extractive Summary: [Extractive Summary] [Format Instruction].
"""

response_list = []
for idx in tqdm(range(len(pubmed))):
    doc = pubmed.iloc[idx, 0]
    prompt = PromptTemplate(template=template, input_variables=["document"])
    llm_chain = LLMChain(prompt=prompt, llm=hf)
    response = llm_chain.invoke(input = doc)['document']
    if len(response) > 0:
        response_list.append([response, pubmed.iloc[idx, 1]])

  8%|▊         | 237/2969 [05:28<1:06:21,  1.46s/it]

In [5]:
for i in response_list:
    print(score.get_Rouge_score(*i))

{'rouge-1': {'r': 0.972972972972973, 'p': 0.4268774703557312, 'f': 0.5934065891675221}, 'rouge-2': {'r': 0.9259259259259259, 'p': 0.3424657534246575, 'f': 0.49999999605800005}, 'rouge-l': {'r': 0.972972972972973, 'p': 0.4268774703557312, 'f': 0.5934065891675221}}
{'rouge-1': {'r': 0.4864864864864865, 'p': 0.07003891050583658, 'f': 0.12244897739159614}, 'rouge-2': {'r': 0.1590909090909091, 'p': 0.015217391304347827, 'f': 0.027777776184177465}, 'rouge-l': {'r': 0.4594594594594595, 'p': 0.06614785992217899, 'f': 0.11564625630316074}}
{'rouge-1': {'r': 0.8035714285714286, 'p': 0.3157894736842105, 'f': 0.4534004997278075}, 'rouge-2': {'r': 0.5654761904761905, 'p': 0.17857142857142858, 'f': 0.27142856778057145}, 'rouge-l': {'r': 0.8035714285714286, 'p': 0.3157894736842105, 'f': 0.4534004997278075}}
{'rouge-1': {'r': 0.9081632653061225, 'p': 1.0, 'f': 0.9518716527655925}, 'rouge-2': {'r': 0.8988095238095238, 'p': 0.967948717948718, 'f': 0.9320987604389576}, 'rouge-l': {'r': 0.9081632653061225